<a href="https://colab.research.google.com/github/tussanakorn/Geospatial-analysis/blob/master/Geospatial_analysis_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download data from google drive (ข้อมูลพิกัดสถานพยาบาลของรัฐ)


In [9]:
!gdown --id 1u-fa0yie-ojxBlq9o_tv8tpONDSv7A7x

Downloading...
From: https://drive.google.com/uc?id=1u-fa0yie-ojxBlq9o_tv8tpONDSv7A7x
To: /content/df_hospital_province.xlsx
100% 107k/107k [00:00<00:00, 31.0MB/s]


Download data from google drive (ข้อมูลอุบัติเหตุทางถนน ปี 2564 (มกราคม - มิถุนายน))

In [10]:
 !gdown --id 1HxahAwd9-iblB8d0me2PVN_s_M58sWLt

Downloading...
From: https://drive.google.com/uc?id=1HxahAwd9-iblB8d0me2PVN_s_M58sWLt
To: /content/accident2020_clean.xlsx
100% 1.88M/1.88M [00:00<00:00, 59.8MB/s]


Install kaggle เพื่อ Download geopandas เพื่อใช้ในการวิเคราะห์ข้อมูล geospatial, folium เพื่อแสดงแผ่นที่

In [3]:
! pip install geopandas
! pip install git+https://github.com/python-visualization/folium

     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 6.3 MB 33.9 MB/s 
     |████████████████████████████████| 15.4 MB 33.3 MB/s 
  Cloning https://github.com/python-visualization/folium to /tmp/pip-req-build-e2f9xfnt
  Running command git clone -q https://github.com/python-visualization/folium /tmp/pip-req-build-e2f9xfnt
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for folium: filename=folium-0.12.1.dev22+g6bde0c2-py2.py3-none-any.whl size=95193 sha256=f18b4d8ff7df974abe3883faffe47c14cae2e2b95fd3363667ce164eabacaac2
  Stored in directory: /tmp/pip-ephem-wheel-cache-0hsgeeyf/wheels/c7/41/34/51d4fc16546a2a77c5ca875d9232ebf13d902dd87615be512c
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
ERROR: pip's dependency resolver

In [11]:
# Import library
import math
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon

import folium
from folium import Choropleth, Marker, GeoJson
from folium.plugins import HeatMap, MarkerCluster

import os
from pathlib import Path

In [12]:
#ฟังก์ชันในการแสดงแผนที่โดยใช้ HTML iframe
from IPython.display import IFrame, HTML

def embed_map(m, file_name):    
    m.save(file_name)
    # Notebook
    # return IFrame(src=file_name, width='100%', height='500px')

    # Colab
    return m

In [13]:
# from google.colab import drive
# drive.mount('/content/gdrive')

## ข้อมูลภูมิศาสตร์ สถานพยาบาลของรัฐ

In [14]:
citizeninfo_health = pd.read_excel("/content/df_hospital_province.xlsx")
citizeninfo_health.head()

,index,รหัสหน่วยงาน,กระทรวง,กรม,หน่วยงาน,หมุดที่ไม่พบความผิดปกติ,ที่อยู่จุดบริการ,จังหวัด,ละติจูด,ลองติจูด
0,0,8899021500001,กระทรวงกลาโหม,กรมแพทย์ทหารบก,โรงพยาบาลค่ายพิชัยดาบหัก,Y,102 หมู่ 8 ต.ท่าเสา อ.เมืองอุตรดิตถ์ จ.อุตรดิต...,อุตรดิตถ์,17.662624,100.138907
1,1,8899021200001,กระทรวงกลาโหม,กรมแพทย์ทหารเรือ,โรงพยาบาลทหารเรือกรุงเทพ,Y,224 ถนนริมทางรถไฟเก่า แขวงบางนา เขตบางนา กรุงเ...,กรุงเทพมหานคร,13.670278,100.588009
2,2,111020400029,กระทรวงกลาโหม,กองทัพบก,กองทัพบก โรงพยาบาลพระมงกุฎเกล้า,Y,แขวงทุ่งพญาไท เขตราชเทวี กรุงเทพมหานคร 10400,กรุงเทพมหานคร,13.767327,100.534169
3,3,113020400030,กระทรวงกลาโหม,กองทัพบก,กองทัพบก โรงพยาบาลอานันทมหิดล,Y,ต.เขาสามยอด อ.เมืองลพบุรี จ.ลพบุรี 15000,ลพบุรี,14.849659,100.666970
4,4,8899020600012,กระทรวงกลาโหม,กองทัพอากาศ,โรงพยาบาลภูมิพลอดุลยเดช กรมแพทย์ทหารอากาศ,Y,แขวงคลองถนน เขตสายไหม กรุงเทพมหานคร 10220,กรุงเทพมหานคร,13.909294,100.618019


In [15]:
citizeninfo_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012 entries, 0 to 1011
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    1012 non-null   int64  
 1   รหัสหน่วยงาน             1012 non-null   int64  
 2   กระทรวง                  1012 non-null   object 
 3   กรม                      1012 non-null   object 
 4   หน่วยงาน                 1012 non-null   object 
 5   หมุดที่ไม่พบความผิดปกติ  1012 non-null   object 
 6   ที่อยู่จุดบริการ         1012 non-null   object 
 7   จังหวัด                  1012 non-null   object 
 8   ละติจูด                  1012 non-null   float64
 9   ลองติจูด                 1012 non-null   float64
dtypes: float64(2), int64(2), object(6)
memory usage: 79.2+ KB


In [16]:
citizeninfo_health.dropna(subset=['ละติจูด', 'ลองติจูด', 'หน่วยงาน'], inplace=True)

In [17]:
citizeninfo_health.isna().sum()

index                      0
รหัสหน่วยงาน               0
กระทรวง                    0
กรม                        0
หน่วยงาน                   0
หมุดที่ไม่พบความผิดปกติ    0
ที่อยู่จุดบริการ           0
จังหวัด                    0
ละติจูด                    0
ลองติจูด                   0
dtype: int64

In [18]:
#create GeoDataFrame
df_hospital = gpd.GeoDataFrame(citizeninfo_health, geometry=gpd.points_from_xy(citizeninfo_health['ลองติจูด'], citizeninfo_health['ละติจูด']))
df_hospital.head()

,index,รหัสหน่วยงาน,กระทรวง,กรม,หน่วยงาน,หมุดที่ไม่พบความผิดปกติ,ที่อยู่จุดบริการ,จังหวัด,ละติจูด,ลองติจูด,geometry
0,0,8899021500001,กระทรวงกลาโหม,กรมแพทย์ทหารบก,โรงพยาบาลค่ายพิชัยดาบหัก,Y,102 หมู่ 8 ต.ท่าเสา อ.เมืองอุตรดิตถ์ จ.อุตรดิต...,อุตรดิตถ์,17.662624,100.138907,POINT (100.13891 17.66262)
1,1,8899021200001,กระทรวงกลาโหม,กรมแพทย์ทหารเรือ,โรงพยาบาลทหารเรือกรุงเทพ,Y,224 ถนนริมทางรถไฟเก่า แขวงบางนา เขตบางนา กรุงเ...,กรุงเทพมหานคร,13.670278,100.588009,POINT (100.58801 13.67028)
2,2,111020400029,กระทรวงกลาโหม,กองทัพบก,กองทัพบก โรงพยาบาลพระมงกุฎเกล้า,Y,แขวงทุ่งพญาไท เขตราชเทวี กรุงเทพมหานคร 10400,กรุงเทพมหานคร,13.767327,100.534169,POINT (100.53417 13.76733)
3,3,113020400030,กระทรวงกลาโหม,กองทัพบก,กองทัพบก โรงพยาบาลอานันทมหิดล,Y,ต.เขาสามยอด อ.เมืองลพบุรี จ.ลพบุรี 15000,ลพบุรี,14.849659,100.666970,POINT (100.66697 14.84966)
4,4,8899020600012,กระทรวงกลาโหม,กองทัพอากาศ,โรงพยาบาลภูมิพลอดุลยเดช กรมแพทย์ทหารอากาศ,Y,แขวงคลองถนน เขตสายไหม กรุงเทพมหานคร 10220,กรุงเทพมหานคร,13.909294,100.618019,POINT (100.61802 13.90929)


In [19]:
hospital = df_hospital[~df_hospital["หน่วยงาน"].str.contains("โรงพยาบาลส่งเสริมสุขภาพตำบล")]
hospital = hospital[hospital["ที่อยู่จุดบริการ"].str.contains("เชียงใหม่") == True]
hospital = hospital[~hospital["หน่วยงาน"].str.contains("จังหวัดแม่ฮ่องสอน")]
hospital = hospital[~hospital["หน่วยงาน"].str.contains("จังหวัดหนองคาย")]
hospital = hospital[~hospital["หน่วยงาน"].str.contains("จังหวัดเชียงราย")]
hospital.head()

,index,รหัสหน่วยงาน,กระทรวง,กรม,หน่วยงาน,หมุดที่ไม่พบความผิดปกติ,ที่อยู่จุดบริการ,จังหวัด,ละติจูด,ลองติจูด,geometry
15,15,114210300023,กระทรวงสาธารณสุข,กรมการแพทย์,กรมการแพทย์ โรงพยาบาลประสาทจังหวัดเชียงใหม่,Y,เลขที่ 2 ถนนสุเทพ ต.สุเทพ อ.เมืองเชียงใหม่ จ.เ...,เชียงใหม่,18.789776,98.969521,POINT (98.96952 18.78978)
27,27,114210800027,กระทรวงสาธารณสุข,กรมสุขภาพจิต,กรมสุขภาพจิต โรงพยาบาลสวนปรุง,Y,ตำบลหายยา อำเภอเมืองเชียงใหม่ จังหวัดเชียงใหม่...,เชียงใหม่,18.780740,98.979153,POINT (98.97915 18.78074)
37,37,8899210200026,กระทรวงสาธารณสุข,สำนักงานปลัดกระทรวงสาธารณสุข,โรงพยาบาลดอยเต่า จังหวัดเชียงใหม่,Y,105 ม.3 ถ.ฮอด-แม่ตื้น หมู่ 3 ต.ดอยเต่า อ.ดอยเต...,เชียงใหม่,17.949973,98.690550,POINT (98.69055 17.94997)
87,87,8899210200037,กระทรวงสาธารณสุข,สำนักงานปลัดกระทรวงสาธารณสุข,โรงพยาบาลแม่วาง จังหวัดเชียงใหม่,Y,ถ.มานีกาด-แม่วาง หมู่ 1 ต.บ้านกาด อ.แม่วาง จ.เ...,เชียงใหม่,18.616122,98.774680,POINT (98.77468 18.61612)
131,131,8899210200027,กระทรวงสาธารณสุข,สำนักงานปลัดกระทรวงสาธารณสุข,โรงพยาบาลดอยสะเก็ด จังหวัดเชียงใหม่,Y,2 ม.8 ถ.เชียงใหม่-ดอยสะเก็ด หมู่ 8 ต.เชิงดอย อ...,เชียงใหม่,18.869617,99.129686,POINT (99.12969 18.86962)


In [21]:
# Create a map
m_2 = folium.Map(location=[15.8700,100.9925], tiles='openstreetmap', zoom_start=5)

# Visualize the hospital locations
for idx, row in hospital.iterrows():    
    Marker([row['ละติจูด'], row['ลองติจูด']], popup=row['หน่วยงาน']).add_to(m_2)
    
# Show the map
m_2

## ข้อมูลภูมิศาสตร์ อุบัติเหตุทางถนน ปี 2564 (มกราคม - มิถุนายน)

In [22]:
collisions = pd.read_excel("/content/accident2020_clean.xlsx")
collisions = gpd.GeoDataFrame(collisions, geometry=gpd.points_from_xy(collisions['LONGITUDE'], collisions['LATITUDE']))

In [23]:
collisions = collisions[(collisions['จังหวัด'] == 'เชียงใหม่' )] 
collisions.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
collisions.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,ปีที่เกิดเหตุ,วันที่เกิดเหตุ,เวลา,วันที่รายงาน,เวลาที่รายงาน,หน่วยงาน,สายทาง,จังหวัด,บริเวณที่เกิดเหตุ/ลักษณะทาง,มูลเหตุสันนิษฐาน,ลักษณะการเกิดอุบัติเหตุ,จำนวนผู้เสียชีวิต,รวมจำนวนผู้บาดเจ็บ,LATITUDE,LONGITUDE,geometry
18,18,2020,2020-01-01,01:10:00,2020-01-02,01:19:00,กรมทางหลวง,ล้องอ้อ - เมืองงาม,เชียงใหม่,ทางตรง+ไม่มีความลาดชัน,เมาสุรา,พลิกคว่ำ/ตกถนนในทางตรง,0,1,19.876908,99.190198,POINT (99.19020 19.87691)
109,109,2020,2020-01-01,11:20:00,2020-01-01,21:18:00,กรมทางหลวง,เชียงใหม่ - บ้านโป่ง,เชียงใหม่,ทางตรง+ไม่มีความลาดชัน,ขับรถเร็วเกินอัตรากำหนด,พลิกคว่ำ/ตกถนนในทางตรง,0,1,18.841415,99.013252,POINT (99.01325 18.84142)
111,111,2020,2020-01-01,11:40:00,2020-01-02,14:33:00,กรมทางหลวง,จอมทอง - ดอยอินทนนท์,เชียงใหม่,ทางโค้งกว้าง+ที่ลาดชัน,ไม่คุ้นเคยเส้นทาง/ขับรถไม่ชำนาญ,พลิกคว่ำ/ตกถนนในทางโค้ง,0,2,18.535118,98.493118,POINT (98.49312 18.53512)
153,153,2020,2020-01-01,13:50:00,2020-01-02,06:11:00,กรมทางหลวงชนบท,แยกทางหลวงหมายเลข 1349 (กม.ที่ 29+000) - บ้านห...,เชียงใหม่,ทางโค้งกว้าง+ที่ลาดชัน,เมาสุรา,อื่นๆ,0,1,18.790448,98.536295,POINT (98.53629 18.79045)
174,174,2020,2020-01-01,15:00:00,2020-01-03,12:14:00,กรมทางหลวง,ล้องอ้อ - เมืองงาม,เชียงใหม่,ทางตรง+ไม่มีความลาดชัน,ขับรถเร็วเกินอัตรากำหนด,พลิกคว่ำ/ตกถนนในทางตรง,0,1,20.040856,99.327811,POINT (99.32781 20.04086)


### แผนที่อุบัติเหตุทางถนน จังหวัดเชียงใหม่

In [24]:
m_3 = folium.Map(location=[15.8700,100.9925], tiles='openstreetmap', zoom_start=5)
HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=15).add_to(m_3)
m_3

## แผนที่โรงพยาบาลในจังหวัดเชียงใหม่ กับบริเวณที่มักเกิดอุบัติเหตุบนท้องถนน

In [25]:
m_4 = folium.Map(location=[15.8700,100.9925], zoom_start=5)

HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=12).add_to(m_4)

# Visualize the hospital locations
for idx, row in hospital.iterrows():    
    Marker([row['ละติจูด'], row['ลองติจูด']], popup=row['หน่วยงาน']).add_to(m_4)    
    
# Show the map
m_4

#GeoSpatial Analysis

In [26]:
hospital.crs = {'init': 'epsg:4240'}
collisions.crs = {'init': 'epsg:4240'}

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [27]:
hospital.crs

<Geographic 2D CRS: +init=epsg:4240 +type=crs>
Name: Indian 1975
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: Thailand - onshore plus offshore Gulf of Thailand.
- bounds: (97.34, 5.63, 105.64, 20.46)
Datum: Indian 1975
- Ellipsoid: Everest 1830 (1937 Adjustment)
- Prime Meridian: Greenwich

In [28]:
collisions.crs

<Geographic 2D CRS: +init=epsg:4240 +type=crs>
Name: Indian 1975
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: Thailand - onshore plus offshore Gulf of Thailand.
- bounds: (97.34, 5.63, 105.64, 20.46)
Datum: Indian 1975
- Ellipsoid: Everest 1830 (1937 Adjustment)
- Prime Meridian: Greenwich

สร้าง Buffer รัศมี 3 กิโลเมตร ขึ้นมารอบ ๆ ทุกโรงพยาบาล

In [29]:
# COV = 3*0.01745329251994328
# coverage = hospital.geometry.buffer(COV) 
# coverage

In [30]:
COV = 3*0.01745329251994328
coverage = hospital.geometry.buffer(COV) 

m =  folium.Map(location=[15.8700,100.9925], zoom_start=6)

HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=12).add_to(m)

for idx, row in hospital.iterrows():
    Marker([row['ละติจูด'], row['ลองติจูด']],popup=row['หน่วยงาน']).add_to(m)
    
# Plot each polygon on the map
GeoJson(coverage.to_crs(epsg=8901)).add_to(m)

embed_map(m, 'm_m.html')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


นำ Coverage มารวมกันเป็น MultiPolygon shape เดียว เพื่อให้ง่ายในการกรอง เอาอุบัติเหตุที่ไม่อยู่ใน shape ออกมา

In [31]:
cov_union = coverage.geometry.unary_union
outside_range = collisions.loc[~collisions["geometry"].apply(lambda x: cov_union.contains(x))]
outside_range.head()

,index,ปีที่เกิดเหตุ,วันที่เกิดเหตุ,เวลา,วันที่รายงาน,เวลาที่รายงาน,หน่วยงาน,สายทาง,จังหวัด,บริเวณที่เกิดเหตุ/ลักษณะทาง,มูลเหตุสันนิษฐาน,ลักษณะการเกิดอุบัติเหตุ,จำนวนผู้เสียชีวิต,รวมจำนวนผู้บาดเจ็บ,LATITUDE,LONGITUDE,geometry
111,111,2020,2020-01-01,11:40:00,2020-01-02,14:33:00,กรมทางหลวง,จอมทอง - ดอยอินทนนท์,เชียงใหม่,ทางโค้งกว้าง+ที่ลาดชัน,ไม่คุ้นเคยเส้นทาง/ขับรถไม่ชำนาญ,พลิกคว่ำ/ตกถนนในทางโค้ง,0,2,18.535118,98.493118,POINT (98.49312 18.53512)
153,153,2020,2020-01-01,13:50:00,2020-01-02,06:11:00,กรมทางหลวงชนบท,แยกทางหลวงหมายเลข 1349 (กม.ที่ 29+000) - บ้านห...,เชียงใหม่,ทางโค้งกว้าง+ที่ลาดชัน,เมาสุรา,อื่นๆ,0,1,18.790448,98.536295,POINT (98.53629 18.79045)
632,632,2020,2020-01-06,11:00:00,2020-01-27,16:05:00,กรมทางหลวง,สะพานแม่กลาง - บ้านบ่อแก้ว,เชียงใหม่,ทางตรง+ไม่มีความลาดชัน,ขับรถเร็วเกินอัตรากำหนด,พลิกคว่ำ/ตกถนนในทางตรง,0,0,18.257928,98.620459,POINT (98.62046 18.25793)
651,651,2020,2020-01-06,20:00:00,2020-01-09,10:27:00,กรมทางหลวง,ปากทางท่าลี่ - สะพานแม่กลาง,เชียงใหม่,ทางตรง+ไม่มีความลาดชัน,ขับรถเร็วเกินอัตรากำหนด,ชนคนเดินเท้า,1,0,18.451639,98.718861,POINT (98.71886 18.45164)
737,737,2020,2020-01-08,18:04:00,2020-03-30,15:11:00,กรมทางหลวง,แม่จา - รินหลวง,เชียงใหม่,ทางโค้งกว้าง+ที่ลาดชัน,อุปกรณ์ยานพาหนะบกพร่อง,พลิกคว่ำ/ตกถนนในทางโค้ง,0,0,19.520016,98.859256,POINT (98.85926 19.52002)


จำนวนอุบัติเหตุที่ไม่อยู่ใน shape

In [32]:
outside_range.shape

(527, 17)

อุบัติเหตุทางถนน ที่อยู่นอกเขต 3 กิโลเมตร จากโรงพยาบาล คิดเป็น 46.03%

In [34]:
percentage = round(100*len(outside_range)/len(collisions), 2)
print("Percentage of collisions more than 3 km away from the closest hospital: {}%".format(percentage))

Percentage of collisions more than 3 km away from the closest hospital: 46.03%


## แนะนำโรงพยาบาลที่ใกล้ที่สุด

ใส่พิกัดที่เกิดอุบัติเหตุเข้าไป ระบบจะแสดงชื่อโรงพยาบาลที่ใกล้ที่สุด



In [35]:
def best_hospital(collision_location):
    # Your code here
    name = hospital.loc[hospital.distance(collision_location).idxmin()]['หน่วยงาน']
    return name

In [36]:
print(best_hospital(outside_range.geometry.iloc[0]))

โรงพยาบาลเทพรัตนเวชชานุกูลเฉลิมพระเกียรติ ๖๐ พรรษา จังหวัดเชียงใหม่


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


## โรงพยาบาลไหนน่าจะงานล้น

พิจารณาเฉพาะอุบัติเหตุที่อยู่นอกเขต 3 กิโลเมตร โรงพยาบาลไหนน่าจะรับ Load มากที่สุด

In [37]:
recommended_hosp = outside_range["geometry"].apply(lambda x: best_hospital(x))
highest_demand = recommended_hosp.value_counts().idxmax()
highest_demand

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


'โรงพยาบาลสันทราย จังหวัดเชียงใหม่'

##สร้างโรงพยาบาลเพิ่ม แถวไหนดี

พล็อตแผนที่ คัดเอาแต่อุบัติเหตุทางถนน ที่อยู่นอกเขตความครอบคลุม แล้วลองเลือกจุด จำนวน 2 จุด เป็นที่สร้างโรงพยาบาลใหม่ ว่าจะทำให้ความคลอบคลุมเพิ่มขึ้นกี่เปอร์เซ็น

In [38]:
m_6 = folium.Map(location=[15.8700,100.9925], zoom_start=5) 

folium.GeoJson(coverage.geometry.to_crs(epsg=8901)).add_to(m_6)
HeatMap(data=outside_range[['LATITUDE', 'LONGITUDE']], radius=12).add_to(m_6)
folium.LatLngPopup().add_to(m_6)

m_6

พิจารณจากแผนที่ด้านบน เราจะลองคลิกลงบนแผนที่ ในย่านที่มีอุบัติเหตุหนาแน่น ที่อยู่นอกวงกลม 2 จุด เพิ่อแสดงพิกัด แล้วเอามาใส่ด้านล่าง เพื่อจุดที่จะสร้างโรงพยาบาลใหม่ ว่าจะทำให้ เปอร์เซ็นอุบัติเหตุที่อยู่นอกเขต ลดลงกี่เปอร์เซ็น

In [39]:
# Proposed location of hospital 1
lat_1 = 18.8319
long_1 =  99.0406

# Proposed location of hospital 2
lat_2 =  18.7481
long_2 =  99.0677

ลองพล็อตแผนที่ หลังจากที่สร้างโรงพยาบาลใหม่ 2 แห่ง ตามพิกัดด้านบน แล้วคำนวนความครอบคลุม เปรียบเทียบกับของเก่า

In [40]:
new_df = pd.DataFrame(
    {'Latitude': [lat_1, lat_2],
    'Longitude': [long_1, long_2]})
new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
new_gdf.crs = {'init' :'epsg:4240'}
new_gdf = new_gdf.to_crs(epsg=8901)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [41]:
# get new percentage
new_coverage = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(COV)
new_my_union = new_coverage.geometry.unary_union
new_outside_range = outside_range.loc[~outside_range["geometry"].apply(lambda x: new_my_union.contains(x))]
new_percentage = round(100*len(new_outside_range)/len(collisions), 2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [42]:
# make the map
m_7 = folium.Map(location=[15.8700,100.9925], zoom_start=5) 
folium.GeoJson(coverage.geometry.to_crs(epsg=8901)).add_to(m_7)
folium.GeoJson(new_coverage.geometry.to_crs(epsg=8901)).add_to(m_7)
for idx, row in new_gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=f'New Hospital {idx+1}', icon=folium.Icon(color='orange')).add_to(m_7)
HeatMap(data=new_outside_range[['LATITUDE', 'LONGITUDE']], radius=12).add_to(m_7)
folium.LatLngPopup().add_to(m_7)
display(embed_map(m_7, 'm_7.html'))

ถ้าเราสร้างโรงพยาบาลใหม่ 2 โรงในตำแหน่งด้านบน อุบัติเหตุนอกเขต 3 กิโลเมตร ลดลงเหลือเพียง 29.61% เท่านั้น

In [44]:
print("(NEW) Percentage of collisions more than 1 km away from the closest hospital: {}%".format(new_percentage))

(NEW) Percentage of collisions more than 1 km away from the closest hospital: 29.61%
